# Wind Data From NOAA's RAP model
---

**NOAA**: National Oceanic and Atmospheric Administration

**RAP**: Rapid Refresh  
Information can be found at the following url:
https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/rapid-refresh-rap

Data can be retrieved using the NetCDF Subset Service (NCSS). Information on this protocol are available at: https://www.unidata.ucar.edu/software/thredds/current/tds/reference/NetcdfSubsetServiceReference.html

The Rapid Refresh (RAP) numerical weather model is run by the National Centers for Environmental Prediction (NCEP), which is part of of the NOAA. Multiple data sources go into the generation of RAP model: commercial aircraft weather data, balloon data, radar data, surface observations, and satellite data. The model generates data down to a 13 km resolution horizontal grid every hour. 

## 1. Sites Location
The location (longitude and latitude) of the wind farms in the Western grid are retrieved.

In [ ]:
import westernintnet
grid = westernintnet.WesternIntNet()

In [ ]:
wind_farm = grid.genbus.groupby('type').get_group('wind')
n_target = len(wind_farm)

print("There are %d wind farms in the Western grid." % n_target)

In [ ]:
wind_farm.head(n=10)

In [ ]:
lon_target = wind_farm.lon.values
lat_target = wind_farm.lat.values
id_target  = wind_farm.index.values
capacity_target = wind_farm.GenMWMax.values

## 2. Wind Data

### A.  Downloading Files from NCEP Server and Filling out Dataframe

In [ ]:
import numpy as np
import pandas as pd
import datetime
import math

The path to all files we will download is created. These are 1 hour resolution files for the year 2016. Note that 2016 is a leap year.

In [ ]:
path = 'https://www.ncei.noaa.gov/thredds/ncss/rap130anl/'

start = datetime.datetime.strptime('2016-01-01', '%Y-%m-%d')
end = datetime.datetime.strptime('2016-12-31', '%Y-%m-%d')
step = datetime.timedelta(days=1)

files = []
while start <= end:
    ts = start.strftime('%Y%m%d')
    url = path + '2016'+ ts[4:6] + '/' + ts + '/'
    for h in range(10000,12400,100):
        files.append(url + 'rap_130_' + ts + '_' + str(h)[1:] + '_000.grb2?')
    start += step

print("There are %d files" % len(files))

The U and V components of the wind speed at 10m and 80 meters are the only variables that will be enclosed in the files. Note that We don't need to consider the entire grid. We retrieve only the variables for a predefined bounding box. The boundaries of the box have been chosen according to the northernmost, easternmost, southernmost and westernmost wind farms.

The data will be downloaded in the NetCDF (Network Common Data Form) format. Instructions given in https://www.unidata.ucar.edu/software/thredds/current/tds/reference/NetcdfSubsetServiceReference.html have been very helpful to access these data.

In [ ]:
# Variables
var = 'var=u-component_of_wind_height_above_ground' + '&' + \
      'var=v-component_of_wind_height_above_ground'

# Bounding Box
box = 'north=49&west=-122&east=-102&south=32&disableProjSubset=on&horizStride=1&addLatLon=true'

# Data Format
extension = 'accept=netCDF'

For each farm, we need to find the closest location on the grid. To do so, we calculate the angular distance between the direction of the wind farm and all the directions of the grid. Two functions are defined on the grid below. The first one, `ll2uv` converts the longitude and latitude of a location to its corresponding unit vector ($x$,$y$,$z$). The second function, `angular_distance`, calculates the scalar product between two vectors and returns the subtended angle.

In [ ]:
def ll2uv(lon, lat):
    cos_lat = math.cos(math.radians(lat))
    sin_lat = math.sin(math.radians(lat))
    cos_lon = math.cos(math.radians(lon))
    sin_lon = math.sin(math.radians(lon))
    
    uv = []
    uv.append(cos_lat * cos_lon)
    uv.append(cos_lat * sin_lon)
    uv.append(sin_lat)
    
    return uv


def angular_distance(uv1, uv2):    
    cos_angle = uv1[0]*uv2[0] + uv1[1]*uv2[1] + uv1[2]*uv2[2]
    if cos_angle >= 1:
        cos_angle = 1
    if cos_angle <= -1:
        cos_angle = -1
    angle = math.degrees(math.acos(cos_angle))
    
    return angle

NREL (National Renewable Energy Laboratory) provides generic power curves to estimate wind power output. Three classes of turbines have been defined, which depends on the available wind speed. An offshore class has also been developed. For each class a power curve is given to convert windspeed at 100m to power output. More information can be found in the following document: https://www.nrel.gov/docs/fy14osti/61714.pdf

Note that we use the **IEC class 2** power curve for all the location here.

In [ ]:
PowerCurves = pd.read_csv('../IECPowerCurves.csv')

def get_power(wspd, turbine):
    match  = (PowerCurves['Speed bin (m/s)'] <= np.ceil(wspd)) & (PowerCurves['Speed bin (m/s)'] >= np.floor(wspd))
    if not match.any():
        return 0
    values = PowerCurves[turbine][match]
    return np.interp(wspd,PowerCurves[turbine][match].index.values,PowerCurves[turbine][match].values)

Data are collected below and a dataframe is filled out. Note that some files are missing. An interpolation will be used later in this notebook.

In [ ]:
import requests
import time
import os
from netCDF4 import Dataset
from collections import OrderedDict
from tqdm import tqdm_notebook

missing = []
target2grid = OrderedDict()
data = pd.DataFrame({'plantID':[], 'U':[], 'V':[], 'Pout':[], 'ts':[], 'tsID':[]})
dt = datetime.datetime.strptime('2016-01-01', '%Y-%m-%d')
step = datetime.timedelta(hours=1)


for i, file in tqdm_notebook(enumerate(files)):
    if i % 1000 == 0: time.sleep(300)
    query = file + var + '&' + box + '&' + extension
    request = requests.get(query)
    
    data_tmp = pd.DataFrame({'plantID':id_target, 'ts':[dt]*n_target, 'tsID':[i+1]*n_target})
    
    if request.status_code == 200:
        with open('tmp.nc', 'wb') as f: 
            f.write(request.content)
        tmp = Dataset('tmp.nc', 'r')
        lon_grid = tmp.variables['lon'][:].flatten()
        lat_grid = tmp.variables['lat'][:].flatten()
        u_wsp = tmp.variables['u-component_of_wind_height_above_ground'][0,1,:,:].flatten()
        v_wsp = tmp.variables['v-component_of_wind_height_above_ground'][0,1,:,:].flatten()
            
        n_grid = len(lon_grid)
        if data.empty:
            # The angular distance is calculated once. The target to grid correspondence is stored in a dictionary.
            for j in range(n_target):
                uv_target = ll2uv(lon_target[j], lat_target[j])
                distance = [angular_distance(uv_target, ll2uv(lon_grid[k],lat_grid[k])) for k in range(n_grid)]                    
                target2grid[id_target[j]] = np.argmin(distance)
         
        data_tmp['U'] = [u_wsp[target2grid[id_target[j]]] for j in range(n_target)]
        data_tmp['V'] = [v_wsp[target2grid[id_target[j]]] for j in range(n_target)]
        data_tmp['Pout'] = np.sqrt(pow(data_tmp['U'],2) + pow(data_tmp['V'],2))
        data_tmp['Pout'] = [get_power(val, 'IEC class 2')*capacity_target[j] for j, val in enumerate(data_tmp['Pout'].values)]
        
        tmp.close()
        os.remove('tmp.nc')
    else:
        print("File %s is missing" % file.split('/')[-1])
        missing.append(file)
        
        # missing data are set to -99.
        data_tmp['U'] = [-99] * n_target
        data_tmp['V'] = [-99] * n_target         
        data_tmp['Pout'] = [-99] * n_target
        
    data = data.append(data_tmp, ignore_index=True, sort=False)   
        
    dt += step

In [ ]:
data['plantID'] = data['plantID'].astype(np.int32)
data['tsID'] = data['tsID'].astype(np.int32)

In [ ]:
data.sort_values(by=['tsID', 'plantID'], inplace=True)
data.reset_index(inplace=True, drop=True)

In [ ]:
data.head(n=10)

The dataframe is saved on disk.

In [ ]:
data.to_pickle('western_wind_output_2016_unfilled.pkl')

### B. Imputing Missing Data

In [ ]:
to_impute = data[data.Pout == -99].index

In [ ]:
dates = pd.DatetimeIndex(data['ts'].values)

For each missing entry, we calculate the mean of the U and V components of the wind speed of all the entries that have the same location (`plantID`), the same month, the same hour and, of course, are missing. Using the derived U and V components of the wind speed, we compute the wind power output.

In [ ]:
for i, j in tqdm_notebook(enumerate(to_impute)):
    if i % len(wind_farm) == 0:
        year, month, day, hour = dates[j].year, dates[j].month, dates[j].day, dates[j].hour
        select = data[(dates.month == month) &  (dates.hour == hour) & (data.Pout != -99)]
    
    k = data.loc[j].plantID
    select = select[select.plantID == k]
    
    data.at[j,'U'] = select['U'].mean()
    data.at[j,'V'] = select['V'].mean()
    data.at[j,'Pout'] = get_power(np.sqrt(data.loc[j].U**2 + data.loc[j].V**2), 'IEC class 2') * wind_farm.loc[k].GenMWMax
    

The data frame is saved on disk. The csv file is the one used by the simulation engine.

In [ ]:
data.to_pickle('western_wind_output_2016.pkl')

In [ ]:
name = "western_wind_output_2016_ts.csv"
data.to_csv(name, header=None, index=False, columns=['tsID','plantID','Pout'])